# Final Capstone Project for IBM Data Science Professional Certificate 

## By Nigel Burrows

### Introduction/Business Problem

The goal of this project is to locate which neighborhoods in Toronto are the best for people that are into fitness. This will be good for gyms and fitness trainers that want to narrow down their focus of which neighborhoods to market to as well as being useful for new immigrants when deciding a neighbourhood to settle down in. This project should address the issue of distance it take for a person to get to a fitness location in our near their neighbourhood. A long commute can be a turn off for many people and so the data provided here will match people that take their health seriosuly with the best neighborhoods for that lifestyle.

### Data 

The data we will use is Foursquare data. This data set will be used to tell which venues are located near certain neighborhoods in Toronto. The Toronto data set was scraped from this Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. That data will be cleaned to remove all records where the Borough is "Not assigned" and all Neighbourhoods will be conjoined if the postal codes are the same

Here is a sample of what that data looks like:

In [4]:
import numpy as np
import pandas as pd

CanadaWikiDfList = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
can_postal = CanadaWikiDfList[0]
can_postal = can_postal[~can_postal.Borough.str.contains("Not assigned")]
can_postal.reset_index(inplace=True, drop=True)
can_postal.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Coupling this with the Foursqaure data that will be retrieved using the Foursquare API we will be able to get data per Borough and see which venues are in the same vicinity